#### Esse código apresenta o modelo semiparamétrico de Cox, Resíduos de Schoenfield e o Resíduos de Martingale. Caso haja interesse em rodar o script em sua máquina, certifique-se de que haja memória RAM suficiente. 


In [ ]:
rm(list=ls())
gc(reset=TRUE)

In [ ]:
library(data.table)
library(survival)
library(tidyverse)

In [ ]:
x <- data.table::fread('srag_covid.csv', sep = ',', header = T)

In [ ]:
#Código pra amostragem caso você opte por rodar uma amostra aleatória
set.seed(123)
x <- sample_n(x, 100000)

In [ ]:
#ETL GERAL - Aplicada em todas os scripts
x|>
  dplyr::filter(classificacao_final == 5 &
                  pcr_sarscov2 == 1 &
                  puerpera  %in% c(2, 9, NA),
                gestante %in% c(5, 6, 9, NA),
                !sexo == 'I',
                ano %in% c(2020, 2021, 2022)) -> x

#ETL que muda/cria/renomeia e condiciona variávies
x|>
  dplyr:: mutate(
    idade = as.numeric(difftime(Sys.Date(), data_nascimento, units = "days")) %/% 365,
    faixa_etaria = dplyr::case_when(
      idade >= 0 & idade <= 4 ~ '0-4',
      idade >= 5 & idade <= 9 ~ '5-9',
      idade >= 10 & idade <= 14 ~ '10-14',
      idade >= 15 & idade <= 19 ~ '15-19',
      idade >= 20 & idade <= 24 ~ '20-24',
      idade >= 25 & idade <= 29 ~ '25-29',
      idade >= 30 & idade <= 34 ~ '30-34',
      idade >= 35 & idade <= 39 ~ '35-39',
      idade >= 40 & idade <= 44 ~ '40-44',
      idade >= 45 & idade <= 49 ~ '45-49',
      idade >= 50 & idade <= 54 ~ '50-54',
      idade >= 55 & idade <= 59 ~ '55-59',
      idade >= 60 & idade <= 64 ~ '60-64',
      idade >= 65 & idade <= 69 ~ '65-69',
      idade >= 70 & idade <= 74 ~ '70-74',
      idade >= 75 & idade <= 79 ~ '75-79',
      idade >= 80 ~ '80+'
    ),
    faixa_etaria_dez = dplyr::case_when(
        idade >= 0   & idade <= 9   ~ '0-9',
        idade >= 10  & idade <= 19  ~ '10-19',
        idade >= 20  & idade <= 29  ~ '20-29',
        idade >= 30  & idade <= 39  ~ '30-39',
        idade >= 40  & idade <= 49  ~ '40-49',
        idade >= 50  & idade <= 59  ~ '50-59',
        idade >= 60  & idade <= 69  ~ '60-69',
        idade >= 70  & idade <= 79  ~ '70-79',
        idade >= 80               ~ '80+',
        TRUE                      ~ 'Sem informação'
    ),
    raca_cor = dplyr::case_when(
      raca_cor == 1 ~ 'Branca',
      raca_cor == 2 ~ 'Preta',
      raca_cor == 3 ~ 'Amarela',
      raca_cor == 4 ~ 'Parda',
      raca_cor == 5 ~ 'Indígena',
      TRUE ~ 'Sem informação'
    ),
    escolaridade = dplyr::case_when(
      escolaridade == 0 ~ 'Sem Escolaridade',
      escolaridade == 1 ~ 'Fundamental 1º Ciclo',
      escolaridade == 2 ~ 'Fundamental 2º Ciclo',
      escolaridade == 3 ~ 'Médio',
      escolaridade == 4 ~ 'Superior',
      TRUE ~ 'Sem informação'
      ),
    possui_cardiopatia = dplyr::case_when(
      possui_cardiopatia == 1 ~ 'Sim',
      possui_cardiopatia == 2 ~ 'Não',
      TRUE ~ 'Sem informação'
    ),
    possui_pneumopatia = dplyr::case_when(
      possui_pneumopatia == 1 ~ 'Sim',
      possui_pneumopatia == 2 ~ 'Não',
      TRUE ~ 'Sem informação'
    ),
    possui_doenca_renal = dplyr::case_when(
      possui_doenca_renal == 1 ~ 'Sim',
      possui_doenca_renal == 2 ~ 'Não',
      TRUE ~ 'Sem informação'
    ),
    possui_doenca_neurologica = dplyr::case_when(
      possui_doenca_neurologica == 1 ~ 'Sim',
      possui_doenca_neurologica == 2 ~ 'Não',
      TRUE ~ 'Sem informação'
    ),
    possui_obesidade = dplyr::case_when(
      possui_obesidade == 1 ~ 'Sim',
      possui_obesidade == 2 ~ 'Não',
      TRUE ~ 'Sem informação'
    ),
    possui_asma = dplyr::case_when(
      possui_asma == 1 ~ 'Sim',
      possui_asma == 2 ~ 'Não',
      TRUE ~ 'Sem informação'
    ),
    tempo_ate_internacao = as.numeric(difftime(data_internacao, data_primeiros_sintomas, units = "days")),
    regiao =  dplyr::case_when(
      sigla_uf_internacao %in% c("AC", "AP", "AM", "PA", "RO", "RR", "TO") ~ "Norte",
      sigla_uf_internacao %in% c("AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE") ~ "Nordeste",
      sigla_uf_internacao %in% c("DF", "GO", "MT", "MS") ~ "Centro-Oeste",
      sigla_uf_internacao %in% c("ES", "MG", "RJ", "SP") ~ "Sudeste",
      sigla_uf_internacao %in% c("PR", "RS", "SC") ~ "Sul",
      TRUE ~ NA_character_
    ),
    evolucao_caso = dplyr::case_when(
      evolucao_caso == 1 ~ 'Cura',
      evolucao_caso == 2 ~ 'Óbito',
      evolucao_caso == 3 ~ 'Óbito por outras causas',
      TRUE ~ NA_character_
    ),
    faixa_etaria_dez = factor(
      faixa_etaria_dez,
      levels = c("0-9", "10-19", "20-29", "30-39", "40-49",
                 "50-59", "60-69", "70-79", "80+")
    ),
    faixa_etaria = factor(
      faixa_etaria,
      levels = c("0-4", "5-9", "10-14", "15-19", "20-24", "25-29",
                 "30-34", "35-39", "40-44", "45-49", "50-54",
                 "55-59", "60-64", "65-69", "70-74", "75-79", "80+")
    )) -> x

#Criação de Variáveis de tempo
x |>
  dplyr::mutate(tempo_ate_obito =
                  as.numeric(difftime(data_alta_obito, data_primeiros_sintomas,
                                      units = 'days')),
                tempo_ate_obito_internacao =
                  as.numeric(difftime(data_alta_obito, data_internacao,
                                      units = 'days')),
                sexo = dplyr::recode(sexo, "F" = "Feminino", "M" = "Masculino")) -> x

#Cria Variável de Status para usar no Kaplan-Meier e Modelo de Cox
x|>
  dplyr::mutate(status =
                  dplyr::case_when(evolucao_caso == 'Óbito' ~ 1,
                                   evolucao_caso == 'Cura' ~ 0)) -> x

In [ ]:
#Filtragem para 60 dias até o óbito
x|>
  dplyr::filter(tempo_ate_obito>=0 &
                  tempo_ate_obito <= 60) -> x

In [ ]:
#Transformação em um objeto Surv
y<-Surv(x$tempo_ate_obito, x$status)

In [ ]:
#Modelagem de Cox
#raca_cor
x$raca_cor <- relevel(as.factor(x$raca_cor), ref = "Branca")
raca_cor<-coxph(y~raca_cor, data=x, x=TRUE)

#Sexo
x$sexo <- relevel(as.factor(x$sexo), ref = 'Feminino')
sexo <- coxph(y~sexo, data=x, x=TRUE)

#Região
x$regiao <- relevel(as.factor(x$regiao), ref = 'Centro-Oeste')
regiao <- coxph(y~regiao, data=x, x=TRUE)

#Ensino
x$escolaridade <- relevel(as.factor(x$escolaridade), ref = 'Superior')
escolaridade <- coxph(y~escolaridade, data=x, x=TRUE)

#Faixas Etárias
x <- x|>
  dplyr::mutate(faixa_etaria_ibge =
                  dplyr::case_when(
                    idade <= 14 ~ 'Infância',
                    idade <= 29 ~ 'Jovem',
                    idade <= 59 ~ 'Adulto',
                    idade >= 60 ~ 'Idoso',
                    TRUE ~ 'Sem informação'))

x$faixa_etaria_ibge <- relevel(as.factor(x$faixa_etaria_ibge), ref = 'Infância')
faixa_etaria <- coxph(y~faixa_etaria_ibge, data=x, x=TRUE)

#Cardiopatia
x$possui_cardiopatia <- relevel(as.factor(x$possui_cardiopatia), ref = 'Não')
cardiopatia <- coxph(y~possui_cardiopatia, data=x, x=TRUE)

#Pneumopatia
x$possui_pneumopatia <- relevel(as.factor(x$possui_pneumopatia), ref = 'Não')
pneumopatia <- coxph(y~possui_pneumopatia, data=x, x=TRUE)

#Doença Renal
x$possui_doenca_renal <- relevel(as.factor(x$possui_doenca_renal), ref = 'Não')
doenca_renal <- coxph(y~possui_doenca_renal, data=x, x=TRUE)

#Doença Neurológica
x$possui_doenca_neurologica <- relevel(as.factor(x$possui_doenca_neurologica), ref = 'Não')
doenca_neurologica <- coxph(y~possui_doenca_neurologica, data=x, x=TRUE)

#Obesidade
x$possui_obesidade <- relevel(as.factor(x$possui_obesidade), ref = 'Não')
obesidade <- coxph(y~possui_obesidade, data=x, x=TRUE)

#Asma
x$possui_asma <- relevel(as.factor(x$possui_asma), ref = 'Não')
asma <- coxph(y~possui_asma, data=x, x=TRUE)

#Modelo final
modelo_final <- coxph(y~faixa_etaria_ibge+
                        escolaridade +
                        regiao +
                        raca_cor +
                        sexo +
                        possui_cardiopatia +
                        possui_pneumopatia +
                        possui_doenca_renal +
                        possui_doenca_neurologica +
                        possui_obesidade +
                        possui_asma,
                      data = x)

In [ ]:
modelo_final

In [ ]:
#residuo Schoenfield
schoenfield <- cox.zph(modelo_final)

In [ ]:
schoenfield

In [ ]:
#boxplot - Schoenfield
data.frame(tempo = schoenfield$time, schoenfield$y) |>
  tidyr::pivot_longer(
    cols = -tempo,
    names_to = "variavel",
    values_to = "residuo"
  ) |>
  dplyr::mutate(faixa_de_tempo = cut(tempo,
                                     breaks = c(0, 15, 30, 45, 60, Inf),
                                     labels = c("0-15 dias", "16-30", "31-45", "46-60", "60+ dias"),
                                     right = TRUE)) |>
  dplyr::filter(!is.na(faixa_de_tempo)) |>
  ggplot(aes(x = faixa_de_tempo, y = residuo)) +
    geom_boxplot(fill = "skyblue", outlier.size = 0.5) +
    geom_hline(yintercept = 0, linetype = "dashed", color = "red") +
    facet_wrap(~ variavel, scales = "free_y", ncol = 3) +
       labs(
      x = "Período de Sobrevida (dias)",
      y = "Resíduo de Schoenfeld"
    ) +
    theme_bw() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1))

In [ ]:
martingale <- residuals(modelo_final, type = "martingale")

In [ ]:
x <- as.data.frame(x)

In [ ]:
dados <- x[names(modelo_final$residuals), ]

In [ ]:
dados$res_martingale <- martingale

In [ ]:
variaveis_categoricas <- c("faixa_etaria_ibge", "escolaridade", "regiao", 
                           "raca_cor", "sexo", "possui_cardiopatia", 
                           "possui_pneumopatia", "possui_doenca_renal",
                           "possui_doenca_neurologica", "possui_obesidade",
                           "possui_asma")

plot_categoricas_boxplot <- dados |>
  tidyr::pivot_longer(
    cols = all_of(variaveis_categoricas),
    names_to = "variavel_categorica",
    values_to = "categoria"
  ) |>
  ggplot(aes(x = categoria, y = res_martingale)) +
    geom_boxplot(fill = "skyblue", outlier.shape = NA) +
    geom_hline(yintercept = 0, linetype = "dashed", color = "red") +
    facet_wrap(~ variavel_categorica, scales = "free", ncol = 3) +
    labs(
      x = "Categorias",
      y = "Resíduo Martingale"
    ) +
    theme_bw() +
    theme(axis.text.x = element_text(angle = 65, hjust = 1))